## 한국전력공사 전기차 충전 데이터셋 이용

In [1]:
import pandas as pd
df = pd.read_csv(r'D:\charging_data\korea_electric_charging_data.csv', encoding='cp949')
print(df.info())

C:\Users\mj\AppData\Local\Temp\ipykernel_16316\3475480627.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\charging_data\korea_electric_charging_data.csv', encoding='cp949')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 14 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   본부      1048575 non-null  object 
 1   사업소     1048575 non-null  object 
 2   충전소명    1048575 non-null  object 
 3   충전기     1048437 non-null  object 
 4   충전기ID   1048575 non-null  object 
 5   주소      1048235 non-null  object 
 6   충전기구분   1048575 non-null  object 
 7   충전용량    992853 non-null   float64
 8   충전량     1048575 non-null  float64
 9   충전시간    1048575 non-null  int64  
 10  충전분     1048575 non-null  int64  
 11  충전종료일자  1048575 non-null  object 
 12  충전시작시각  1048575 non-null  object 
 13  충전종료시각  1048575 non-null  object 
dtypes: float64(2), int64(2), object(10)
memory usage: 112.0+ MB
None


#### 필요없는 열 삭제하기

In [2]:
df = df.drop('본부', axis=1)
df = df.drop('사업소', axis=1)
df = df.drop('충전기', axis=1)
df = df.drop('주소', axis=1)
df = df.drop('충전량', axis=1)

#### 급속 데이터만 추출하기

In [3]:
# 급속데이터만 분리
df = df[df['충전기구분']=="급속"]

# 충전소명을 기준으로 데이터 수 확인 
df['충전기구분'].value_counts()

충전기구분
급속    631047
Name: count, dtype: int64

#### 충전일을 기반으로 날짜 구하기

In [4]:
df['충전종료일자_v2'] = pd.to_datetime(df['충전종료일자'])

# 88일 차이
df['충전종료일자_v2'].max() - df['충전종료일자_v2'].min()


Timedelta('88 days 00:00:00')

#### 충전 세션 수가 700회 이하인 충전기 제거 (88일동안 700회)

In [5]:
# 이상치 데이터 삭제
filter = df['충전기ID'].value_counts()

# 10000보다 작은 loc 추출
to_drop = filter[filter < 700].index

# to_drop에 들어있지 않은 loc만 추출
df = df[~df['충전기ID'].isin(to_drop)]

df['충전기ID'].value_counts()

충전기ID
9813     2671
9812     2422
9814     2285
8847     2031
8848     1954
10900    1613
8850     1494
8849     1442
10618    1313
10418    1256
10394    1229
10419    1205
8043     1160
10420    1084
10959    1079
10955    1038
628      1025
631      1023
10901    1021
11        962
10958     957
629       948
10417     943
630       942
632       869
6797      853
5927      838
10392     835
10393     833
8135      830
8042      793
5867      787
10957     784
6756      775
829       765
10614     764
1221      762
1496      756
828       745
1237      737
611       731
645       726
8777      720
6086      716
1497      709
10619     709
1173      709
Name: count, dtype: int64

#### 충전 시간이 3시간 이상인 데이터 제거

In [6]:
# 충전 시간 (hour) 확인 후 이상치 제거
filter = df['충전시간'].value_counts()

to_drop = filter[filter <= 3].index
df = df[~df['충전시간'].isin(to_drop)]

df['충전시간'].value_counts()

충전시간
0     40934
1      8903
2       972
3        18
13        4
Name: count, dtype: int64

#### 충전 분이 5분 이하인 데이터 제거

In [7]:

df = df[df['충전분'] >= 5]

df['충전분'].value_counts()

충전분
36    1367
38    1323
37    1312
39    1289
40    1241
35    1239
34    1209
32    1166
33    1159
30    1075
41    1073
31    1065
29    1037
42    1014
28     999
23     982
20     979
43     971
24     958
25     956
21     949
44     942
26     938
19     937
22     931
27     928
18     920
47     886
48     846
16     839
46     830
17     825
45     817
15     804
50     760
49     760
52     724
51     707
14     693
13     667
12     652
10     642
11     628
9      622
54     616
53     595
55     577
56     560
57     547
8      541
59     530
7      504
5      497
6      491
58     489
Name: count, dtype: int64

In [8]:
df['충전소명'].value_counts()

충전소명
대평동 BRT전기버스 차고지    9417
종합경기장              4474
장등공용차고지            4277
광주시 전기버스 충전소       3765
첨단공용차고지            3575
회차지(월드컵경기장)        2774
매월동 전기버스 충전소       2722
월남공용차고지            1948
빛가람동 공용차고지         1934
뚝도충전소              1351
대전광역시청 부설주차장       1340
이천휴게소(하남방향)         908
을숙도 공영주차장           770
계양구청                757
인천업사이클 에코센터         752
도산버스차고지             737
부산대북측               719
강남구청 공영주차장          715
하나로마트 진접점           696
개인택시조합              678
상암월드컵경기장            674
도봉구청                664
가락2동 주민센터           661
LH서울지사              653
만덕2동 주민센터           647
Name: count, dtype: int64

In [9]:
# 요일을 0-월 1-화 ... 6-일 로 맵핑 (datetime 객체 이용)

df['date'] = pd.to_datetime(df['date'])

df['date'] = df['date'].dt.weekday

df['date'].value_counts()

KeyError: 'date'

In [ ]:
# 주말인지 아닌지 여부 데이터 추가

def is_weekend(day):
    if day in [0,1,2,3,4]:
        return 0
    else:
        return 1

df['weekend'] = df['date'].apply(is_weekend)


In [ ]:
# 하루를 10분 단위의 144개로 나누어, 충전 시작 시간에 맞게 할당
def time_calculate(time):
    hour, minute = time.split()[1].split(':')
    total_minutes = int(hour) * 60 + int(minute)
    time_interval = total_minutes // 10 
    return time_interval

df['time'] = df['start_time'].apply(time_calculate)

In [ ]:
#새로운 df 구성


charging_df = df.iloc[:, [8, 7, 4, 1]]
charging_df = charging_df.drop_duplicates()

# id 별 중복 값이 너무 많은 것인지 id당 3개월에 하루 1세션도 안됨.
charging_df["id"].value_counts()